<a href="https://colab.research.google.com/github/achrisk/Dissertation/blob/main/within_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/sagihaider/EEG_Deep.git

Cloning into 'EEG_Deep'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 517 (delta 22), reused 0 (delta 0), pack-reused 477
Receiving objects: 100% (517/517), 1.69 GiB | 30.16 MiB/s, done.
Resolving deltas: 100% (252/252), done.
Checking out files: 100% (95/95), done.


In [3]:
import scipy.io as spio
import numpy as np
import pandas as pd
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from EEG_Deep.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

In [4]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [5]:
from numpy import zeros
K.clear_session()

X_tr = np.empty([80, 12, 4096])
X_ts = np.empty([40, 12, 4096])
result=[]

h_cut = [24]
drop_out = [0.25]
k_len = [32, 64, 128]
n_epochs = 500

outfname = 'accuray_epochs' + str(n_epochs) + '_filter_' + str(h_cut) + '_drop_' + str(drop_out) + '_patient_data.npy'

nsub = 10
nfilt = len(h_cut)
ndrop = len(drop_out)
nkl = len(k_len)
acc_sub = zeros([nsub, nfilt,ndrop,nkl])

for sub_idx, x in enumerate(range(1,11)):
    for h_indx, h in enumerate(h_cut):
        fName = 'EEG_Deep/Data2A/parsed_P0' + str(x) + 'T.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_tr = mat['RawEEGData']
        y_tr = mat['Labels']
        y_tr = y_tr.flatten() 

        print(np.shape(r_X_tr))
        print(np.shape(y_tr))

        for t in range(r_X_tr.shape[0]):
            tril = r_X_tr[t,:,:]
            #tril = tril.transpose()
            tril_filtered = butter_bandpass_filter(tril, lowcut=8, highcut=h, fs=250, order=4)
            # tril_filtered = tril_filtered.transpose()
            X_tr[t,:,:] = tril_filtered 

            # split data of each subject in training and validation
        X_train      = X_tr[0:60,:,2048:3584]
        Y_train      = y_tr[0:60]
        X_val       = X_tr[60:,:,2048:3584]
        Y_val       = y_tr[60:]

        print(np.shape(X_train))
        print(np.shape(Y_train))
        print(np.shape(X_val))
        print(np.shape(Y_val))

        # convert labels to one-hot encodings.
        Y_train = np_utils.to_categorical(Y_train-1, num_classes=2)
        Y_val = np_utils.to_categorical(Y_val-1, num_classes=2)

        kernels, chans, samples = 1, 12, 1536
        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], kernels, chans, samples)
        X_val       = X_val.reshape(X_val.shape[0], kernels, chans, samples)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_val.shape[0], 'val samples')

        fName = 'EEG_Deep/Data2A/parsed_P0' + str(x) + 'E.mat'  # Load Data
        print(fName)
        mat = spio.loadmat(fName)
        r_X_ts = mat['RawEEGData']
        y_ts = mat['Labels']
        y_ts = y_ts.flatten() 

        print(np.shape(r_X_ts))
        print(np.shape(y_ts))

        for t in range(r_X_ts.shape[0]):
            tril = r_X_ts[t,:,:]
            # tril = tril.transpose()
            tril_filtered = butter_bandpass_filter(tril, lowcut=8, highcut=h, fs=250, order=4)
            # tril_filtered = tril_filtered.transpose()
            X_ts[t,:,:] = tril_filtered 

        X_test      = X_ts[:,:,2048:3584]
        Y_test      = y_ts[:]
        print(np.shape(X_test))
        print(np.shape(Y_test))

        #convert labels to one-hot encodings.
        Y_test      = np_utils.to_categorical(Y_test-1)

        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

        print('X_train shape:', X_test.shape)
        print(X_test.shape[0], 'test samples')

        kernels, chans, samples = 1, 12, 1536
        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_train = X_train.reshape(X_train.shape[0], kernels, chans, samples)
        X_val   = X_val.reshape(X_val.shape[0], kernels, chans, samples)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_val.shape[0], 'val samples')

        X_test      = X_ts[:,:,2048:3584]
        Y_test      = y_ts[:]
        print(np.shape(X_test))
        print(np.shape(Y_test))

        #convert labels to one-hot encodings.
        Y_test      = np_utils.to_categorical(Y_test-1, num_classes=2)

        # convert data to NCHW (trials, kernels, channels, samples) format. Data 
        # contains 22 channels and 500 time-points. Set the number of kernels to 1.
        X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

        print('X_train shape:', X_test.shape)
        print(X_test.shape[0], 'train samples')

        # acc_sub = zeros([rows, cols])
        for id_d, d in enumerate(drop_out):
            for id_kl, kl in enumerate(k_len):
              print(id_kl, id_d)
              # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
              # model configurations may do better, but this is a good starting point)
              model = EEGNet(nb_classes = 2, Chans = 12, Samples = 1536,
                             dropoutRate = d, kernLength = kl, F1 = 8,D = 2, F2 = 16,
                             norm_rate = 0.25, dropoutType = 'Dropout')

              # compile the model and set the optimizers
              model.compile(loss='categorical_crossentropy', optimizer='adam', 
                            metrics = ['accuracy'])

              # count number of parameters in the model
              numParams    = model.count_params() 

              # set a valid path for your system to record model checkpoints
              checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

              # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
              # the weights all to be 1
              class_weights = {0:1, 1:1}

              history = model.fit(X_train, Y_train, batch_size = 16, epochs = n_epochs, 
                                  verbose = 2, validation_data=(X_val, Y_val),
                                  callbacks=[checkpointer], class_weight = class_weights)

              print('\n# Evaluate on test data')
              results = model.evaluate(X_test, Y_test, batch_size=1)
              print('test loss, test acc:', results)

              acc_sub[sub_idx,h_indx,id_d, id_kl] = results[1]

              from keras import backend as K 
              # Do some code, e.g. train and save model
              K.clear_session()
              
np.save(outfname, acc_sub)


EEG_Deep/Data2A/parsed_P01T.mat
(80, 12, 4096)
(80,)
(60, 12, 1536)
(60,)
(20, 12, 1536)
(20,)
X_train shape: (60, 1, 12, 1536)
60 train samples
20 val samples
EEG_Deep/Data2A/parsed_P01E.mat
(40, 12, 4096)
(40,)
(40, 12, 1536)
(40,)
X_train shape: (40, 1, 12, 1536)
40 test samples
X_train shape: (60, 1, 12, 1536)
60 train samples
20 val samples
(40, 12, 1536)
(40,)
X_train shape: (40, 1, 12, 1536)
40 train samples
0 0
Epoch 1/500

Epoch 00001: val_loss improved from inf to 0.83718, saving model to /tmp/checkpoint.h5
4/4 - 0s - loss: 0.6877 - accuracy: 0.5667 - val_loss: 0.8372 - val_accuracy: 0.0000e+00
Epoch 2/500

Epoch 00002: val_loss did not improve from 0.83718
4/4 - 0s - loss: 0.5974 - accuracy: 0.6667 - val_loss: 0.9586 - val_accuracy: 0.0000e+00
Epoch 3/500

Epoch 00003: val_loss did not improve from 0.83718
4/4 - 0s - loss: 0.5780 - accuracy: 0.6667 - val_loss: 1.0217 - val_accuracy: 0.0000e+00
Epoch 4/500

Epoch 00004: val_loss did not improve from 0.83718
4/4 - 0s - loss: 0

KeyboardInterrupt: ignored